## Parte 1: Preparación de Datos y Definición del Área de Estudio

### Introducción
Este proyecto implementa metodologías de detección de cambios usando imágenes Sentinel-1 SAR para identificar cambios potenciales asociados a actividad agrícola en municipios seleccionados de los departamentos de Casanare y Meta, Colombia [1]. La detección de cambios mediante radar de apertura sintética (SAR) ofrece ventajas significativas sobre sensores ópticos, particularmente en regiones tropicales con cobertura nubosa frecuente [2].

### Área de estudio
Los municipios seleccionados corresponden a zonas de importancia agrícola en la región de la Orinoquía colombiana:

**Departamento del Meta:**
- Puerto López
- Castilla La Nueva
- San Carlos de Guaroa
- Cabuyaro

**Departamento de Casanare:**
- Tauramena
- Yopal
- Aguazul
- Nunchía
- Villanueva

### Metodología
Este notebook implementa la primera fase del procesamiento siguiendo la metodología propuesta por Canty et al. [3] para análisis estadístico de cambios en series temporales de Sentinel-1 en Google Earth Engine. Los datos SAR de polarización dual (VV y VH) serán utilizados para detectar cambios estadísticamente significativos en la superficie terrestre.

## 1. Carga de librerías

Trabaja dentro de un entorno virtual dedicado para mantener controladas las dependencias del proyecto. Activa el entorno y asegúrate de que las bibliotecas de análisis geoespacial, visualización y la API de Google Earth Engine estén instaladas antes de ejecutar las celdas. Estas herramientas permiten replicar los ejercicios de cambio multitemporal descritos en el tutorial oficial [4]. Importa los módulos siguiendo el orden estándar → científicos → geoespaciales → visualización para que cualquiera identifique las dependencias de un vistazo.

In [ ]:
# Instalación de dependencias necesarias
# !pip install earthengine-api geemap geopandas fiona

import ee
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

print("Librerías cargadas exitosamente")

## 2. Inicialización de Google Earth Engine

Earth Engine es el motor que nos permite descargar y procesar las imágenes Sentinel-1. Usa el bloque `try/except` que llama primero a `ee.Initialize()` y, si falla, ejecuta `ee.Authenticate()` seguido de `ee.Initialize()`. Este patrón evita interrupciones cuando la sesión expira y es el que recomienda el tutorial de cambios con Sentinel-1 [4].

In [ ]:
# Inicializar Earth Engine
try:
    ee.Initialize()
    print("Earth Engine inicializado correctamente")
except:
    print("Autenticando Earth Engine...")
    ee.Authenticate()
    ee.Initialize()
    print("Earth Engine autenticado e inicializado")

## 3. Carga de la capa de municipios desde Earth Engine

Cargamos la FeatureCollection `users/geoia/MapasBase/ColMuni` que contiene los límites municipales de Colombia disponibles directamente en Earth Engine. Esta capa contiene los nueve municipios seleccionados en Casanare y Meta, que representan la diversidad agrícola de los Llanos Orientales según los reportes estadísticos del sector [5].

In [ ]:
# FeatureCollection de Earth Engine con municipios de Colombia
ee_asset = 'users/geoia/MapasBase/ColMuni'

# Cargar la FeatureCollection completa
municipios_colombia = ee.FeatureCollection(ee_asset)

# Obtener información básica
n_total = municipios_colombia.size().getInfo()
print(f"Total de municipios en Colombia (EE): {n_total}")

# Mostrar las primeras propiedades
sample_props = municipios_colombia.first().propertyNames().getInfo()
print(f"\nPropiedades disponibles: {sample_props}")

## 4. Selección de municipios de interés

Filtra la capa con los códigos DANE de cada municipio y guarda la lista en una variable clara (`municipios_objetivo`). Esta lista sirve para repetir el análisis cuando se actualicen los datos y facilita que otros revisen la configuración sin buscar manualmente los filtros.

In [ ]:
# Definir los municipios objetivo usando códigos MPIO_CCDGO
codigos_municipios = [
    '50573',  # META - PUERTO LÓPEZ
    '50150',  # META - CASTILLA LA NUEVA
    '50680',  # META - SAN CARLOS DE GUAROA
    '50124',  # META - CABUYARO
    '85410',  # CASANARE - TAURAMENA
    '85001',  # CASANARE - YOPAL
    '85010',  # CASANARE - AGUAZUL
    '85225',  # CASANARE - NUNCHÍA
    '85440'   # CASANARE - VILLANUEVA
]

print("Códigos de municipios seleccionados:")
for codigo in codigos_municipios:
    print(f"  {codigo}")

In [ ]:
# Filtrar los municipios de interés usando Earth Engine
municipios_seleccionados = municipios_colombia.filter(
    ee.Filter.inList('MPIO_CCDGO', codigos_municipios)
)

n_seleccionados = municipios_seleccionados.size().getInfo()
print(f"\nMunicipios encontrados: {n_seleccionados}")

# Obtener información detallada
municipios_info = municipios_seleccionados.select(['DPTO_CNMBR', 'MPIO_CNMBR', 'MPIO_CCDGO', 'MPIO_NAREA']).getInfo()

print("\nDetalle de municipios:")
for feat in municipios_info['features']:
    props = feat['properties']
    print(f"  {props['MPIO_CNMBR']:<25} ({props['DPTO_CNMBR']:<10}) - Código: {props['MPIO_CCDGO']}, Área: {props.get('MPIO_NAREA', 'N/A')} km²")

## 5. Cálculo de geometrías y metadatos

Calculamos el centroide y bounds directamente en Earth Engine. Estos valores se usan para centrar mapas y definir el área de análisis.

In [ ]:
# Crear geometría única (unión de todos los municipios)
aoi = municipios_seleccionados.geometry()

# Calcular área total
area_total_m2 = aoi.area().getInfo()
area_total_km2 = area_total_m2 / 1e6

# Calcular centroide
centroid = aoi.centroid().coordinates().getInfo()
centroide_lon = centroid[0]
centroide_lat = centroid[1]

# Calcular bounding box
bounds_geom = aoi.bounds().getInfo()

print(f"\nÁrea total: {area_total_km2:,.2f} km²")
print(f"Centroide del área de estudio: Lat {centroide_lat:.4f}, Lon {centroide_lon:.4f}")
print(f"\nBounding Box (GeoJSON): {bounds_geom}")

## 6. Verificación del AOI de Earth Engine

El AOI (Area of Interest) ya está creado como una geometría de Earth Engine lista para ser usada en los análisis posteriores. No requiere conversión adicional.

In [ ]:
# El AOI ya está en formato Earth Engine
print("✓ AOI creado directamente desde FeatureCollection de Earth Engine")
print(f"  Tipo: {type(aoi)}")
print(f"  Número de municipios: {n_seleccionados}")

# El aoi_geometry es la geometría unificada
aoi_geometry = aoi
print("✓ Geometría unificada lista para análisis")

## 7. Definición de parámetros temporales

Define `start_date` y `end_date` para cubrir varios años agrícolas. Las series largas suavizan el efecto de semanas nubladas y permiten detectar tendencias estacionales, tal como se explica en los ejercicios de Sentinel-1 con Earth Engine [4].

In [ ]:
# Definir período de análisis - Segundo Semestre 2025
# Se recomienda usar al menos 1 año de datos para capturar ciclos agrícolas completos

# Fecha de inicio y fin (ajustar según necesidades)
fecha_inicio = '2025-01-01'  # Formato: YYYY-MM-DD
fecha_fin = '2025-09-30'

# Convertir a objetos datetime
start_date = datetime.strptime(fecha_inicio, '%Y-%m-%d')
end_date = datetime.strptime(fecha_fin, '%Y-%m-%d')

print(f"Período de análisis:")
print(f"  Inicio: {start_date.strftime('%Y-%m-%d')}")
print(f"  Fin: {end_date.strftime('%Y-%m-%d')}")
print(f"  Duración: {(end_date - start_date).days} días")

# Guardar parámetros para uso en notebooks posteriores
parametros = {
    'fecha_inicio': fecha_inicio,
    'fecha_fin': fecha_fin,
    'ee_asset': ee_asset,
    'codigos_municipios': codigos_municipios,
    'centroide_lat': centroide_lat,
    'centroide_lon': centroide_lon,
    'bounds': bounds_geom,
    'area_km2': area_total_km2
}

print("\nParámetros definidos y guardados")

## 8. Visualización del área de estudio

Usa geemap para dibujar los municipios seleccionados sobre un mapa base. Esta vista rápida confirma que las geometrías son correctas y ayuda a comunicar la zona de trabajo a públicos no técnicos. Puedes añadir etiquetas o marcadores si cuentas con parcelas de referencia.

In [ ]:
# Crear mapa interactivo con geemap
Map = geemap.Map(center=[centroide_lat, centroide_lon], zoom=9)

# Agregar la capa de municipios desde Earth Engine
Map.addLayer(municipios_seleccionados, {'color': 'red'}, 'Municipios de Estudio', True, 0.5)

# Agregar el AOI (geometría unificada)
Map.addLayer(aoi, {'color': 'blue'}, 'AOI - Área Unificada', True, 0.3)

# Mostrar mapa
Map

## 9. Exportar parámetros para notebooks posteriores

Guardamos solo los parámetros en formato JSON. Las geometrías se reconstruirán directamente desde Earth Engine en los notebooks posteriores usando el asset y los códigos de municipios.

In [ ]:
# Guardar parámetros en formato JSON
import json
with open('data/parametros.json', 'w') as f:
    json.dump(parametros, f, indent=2)

print(f"✓ Parámetros guardados en: data/parametros.json")
print(f"\nContenido:")
print(f"  - Período: {fecha_inicio} a {fecha_fin}")
print(f"  - Asset EE: {ee_asset}")
print(f"  - Municipios: {len(codigos_municipios)}")
print(f"  - Centroide: [{centroide_lat:.4f}, {centroide_lon:.4f}]")
print(f"  - Área total: {area_total_km2:.2f} km²")

## Resumen

Este notebook prepara la zona de estudio y los parámetros temporales que usarán los análisis posteriores. Entregamos límites municipales en WGS84, archivos listos para Earth Engine y un mapa de verificación que ayuda a alinear al equipo.

**Próximo notebook:** `02_preprocesamiento_sentinel1.ipynb` limpia y organiza la serie Sentinel-1 antes de calcular indicadores de cambio.

### Referencias bibliográficas

[1] M. J. Canty, A. A. Nielsen, H. Skriver y K. Conradsen, “Statistical Analysis of Changes in Sentinel-1 Time Series on the Google Earth Engine,” Remote Sensing, vol. 12, no. 1, p. 46, 2020, doi: 10.3390/rs12010046.

[2] J. Reiche et al., “Combining Satellite Data for Better Tropical Forest Monitoring,” Nature Climate Change, vol. 6, no. 2, pp. 120–122, 2016, doi: 10.1038/nclimate2919.

[3] M. J. Canty, *Image Analysis, Classification and Change Detection in Remote Sensing, with Algorithms for Python*, 4.ª ed., Boca Raton, FL, USA: CRC Press, 2019.

[4] Google Earth Engine Developers, “Detecting Changes in Sentinel-1 Imagery, Part 1,” Google Developers, Mountain View, CA, USA, 2024. Disponible en: https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-1

[5] Ministerio de Agricultura y Desarrollo Rural, *Evaluación Agropecuaria del Departamento del Meta y Casanare*, Bogotá, Colombia, 2023.

[6] European Space Agency, *Sentinel-1 User Handbook*, ESA-EOEP-CSCOP-TN-13-0001, Noordwijk, Países Bajos, 2013.